# Hometown - Analytics Case
## Pipeline Principal - Extração de Dados SIGEL/ANEEL

Este notebook contém o pipeline principal para extração de dados de aerogeradores.

## 1. Setup e Imports

In [ ]:
import sys
from pathlib import Path
import pandas

# Adicionar src ao path
project_root = Path.cwd().parent
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

# Imports do projeto
from extraction.extractors import SigelExtractor
from extraction.validators import (
    validate_api_connection, 
    validate_response_structure,
    validate_extraction_results
)
from config.settings import SIGEL_CONFIG
from utils.logger import setup_logger
from utils.exceptions import APIConnectionError, ValidationError

# Setup logger
logger = setup_logger(__name__, "extraction.log")

print("Setup concluído!")
print(f"Projeto root: {project_root}")
print(f"URL da API: {SIGEL_CONFIG['url']}")

## 2. Validação da Conectividade

In [ ]:
# Testar conexão com a API
try:
    logger.info("Iniciando validação de conectividade...")
    
    api_url = SIGEL_CONFIG["url"]
    connection_ok = validate_api_connection(api_url)
    
    if connection_ok:
        print("✅ API conectada com sucesso!")
        logger.info("Validação de conectividade concluída com sucesso")
    else:
        print("❌ Falha na conexão com API")
        
except (APIConnectionError, ValidationError) as e:
    print(f"❌ Erro de validação: {e}")
    logger.error(f"Erro de validação: {e}")
except Exception as e:
    print(f"❌ Erro inesperado: {e}")
    logger.error(f"Erro inesperado durante validação: {e}")

## 3. Extração de Dados

In [ ]:
# Executar extração completa
try:
    logger.info("Iniciando extração de dados...")
    print("🚀 Iniciando extração de dados dos aerogeradores...")
    
    # Instanciar extrator
    extractor = SigelExtractor()
    
    # Executar extração
    saved_files = extractor.extract_all_data()
    
    print(f"\n✅ Extração concluída!")
    print(f"📁 Arquivos salvos: {len(saved_files)}")
    
    # Mostrar alguns arquivos salvos
    print("\n📋 Primeiros arquivos salvos:")
    for i, file_path in enumerate(saved_files[:5]):
        print(f"  {i+1}. {Path(file_path).name}")
    
    if len(saved_files) > 5:
        print(f"  ... e mais {len(saved_files) - 5} arquivos")
        
    logger.info(f"Extração concluída com sucesso. {len(saved_files)} arquivos salvos")
    
except Exception as e:
    print(f"❌ Erro durante extração: {e}")
    logger.error(f"Erro durante extração: {e}")
    raise

## 4. Validação dos Resultados

In [ ]:
# Validar resultados da extração
try:
    if 'saved_files' in locals() and saved_files:
        logger.info("Iniciando validação dos resultados...")
        print("🔍 Validando resultados da extração...")
        
        # Validação básica (expandir posteriormente)
        validation_ok = validate_extraction_results(
            saved_files=saved_files,
            expected_records=0  # TODO: usar contagem real
        )
        
        if validation_ok:
            print("✅ Validação dos resultados concluída!")
            logger.info("Validação dos resultados concluída com sucesso")
        else:
            print("❌ Falha na validação dos resultados")
            
    else:
        print("⚠️ Nenhum arquivo para validar. Execute a extração primeiro.")
        
except Exception as e:
    print(f"❌ Erro durante validação: {e}")
    logger.error(f"Erro durante validação dos resultados: {e}")

## 5. Resumo da Execução

In [ ]:
# Mostrar resumo final
if 'saved_files' in locals():
    print("📊 RESUMO DA EXTRAÇÃO")
    print("=" * 40)
    print(f"🗂️  Total de arquivos: {len(saved_files)}")
    
    # Calcular tamanho total dos arquivos
    total_size = 0
    for file_path in saved_files:
        if Path(file_path).exists():
            total_size += Path(file_path).stat().st_size
    
    print(f"💾 Tamanho total: {total_size / 1024 / 1024:.2f} MB")
    print(f"📁 Localização: data/raw/")
    print(f"🕒 Padrão de nomes: aerogeradores_raw_YYYYMMDD_HHMMSS_page_XXXX.json")
    
    # Status final
    print("\n🎉 Extração de dados concluída com sucesso!")
    print("🔄 Próximo passo: Transformação dos dados (JSON → Parquet)")
    
else:
    print("⚠️ Nenhuma extração foi executada nesta sessão.")

## 6. Debug e Exploração (Opcional)

In [ ]:
# Explorar estrutura de um arquivo extraído
import json

if 'saved_files' in locals() and saved_files:
    # Ler primeiro arquivo como exemplo
    first_file = saved_files[0]
    print(f"🔍 Explorando estrutura do arquivo: {Path(first_file).name}")
    
    with open(first_file, 'r', encoding='utf-8') as f:
        sample_data = json.load(f)
    
    print(f"\n📋 Estrutura do JSON:")
    print(f"  - Chaves principais: {list(sample_data.keys())}")
    
    if 'features' in sample_data:
        features = sample_data['features']
        print(f"  - Total de features: {len(features)}")
        
        if features:
            first_feature = features[0]
            print(f"  - Estrutura da feature:")
            print(f"    - Chaves: {list(first_feature.keys())}")
            
            if 'attributes' in first_feature:
                attrs = first_feature['attributes']
                print(f"    - Attributes: {list(attrs.keys())[:10]}...")  # Primeiros 10
            
            if 'geometry' in first_feature:
                geom = first_feature['geometry']
                print(f"    - Geometry type: {geom.get('type', 'N/A')}")
else:
    print("⚠️ Nenhum arquivo disponível para exploração.")

In [ ]:
# Teste de import
try:
    from transformation.processors import DataProcessor
    from transformation.geo_utils import extract_coordinates, validate_geometry
    print("✅ Imports da transformação OK!")
except Exception as e:
    print(f"❌ Erro no import: {e}")

# Teste básico
processor = DataProcessor()
json_files = processor.discover_raw_files()
print(f"📁 Arquivos JSON encontrados: {len(json_files)}")

# Testar um arquivo só
if json_files:
    print(f"🔍 Testando: {json_files[0].name}")
    test_result = processor.process_single_file(json_files[0])
    print(f"✅ Resultado: {test_result}")

In [ ]:
# Processar todos os 24 arquivos JSON → Parquet
print("🚀 Processando todos os arquivos...")
all_results = processor.process_all_files()

print(f"\n📊 RESULTADO FINAL:")
print(f"✅ Arquivos processados: {len(all_results)}")

# Verificar tamanho total
total_size = 0
for file_path in all_results:
    if Path(file_path).exists():
        total_size += Path(file_path).stat().st_size

print(f"💾 Tamanho total Parquets: {total_size / 1024 / 1024:.2f} MB")
print(f"🗂️ Localização: data/processed/")

In [ ]:
# Teste de import
try:
    from consolidation.consolidators import DataConsolidator
    print("✅ Import da consolidação OK!")
except Exception as e:
    print(f"❌ Erro no import: {e}")

# Instanciar consolidador
consolidator = DataConsolidator()

# Descobrir arquivos Parquet
parquet_files = consolidator.discover_parquet_files()
print(f"📁 Arquivos Parquet encontrados: {len(parquet_files)}")

# Executar consolidação completa
print("🚀 Iniciando consolidação completa...")
output_path = consolidator.consolidate_all()

print(f"\n✅ Consolidação concluída!")
print(f"📄 Arquivo CSV: {Path(output_path).name}")
print(f"📁 Localização: {output_path}")

In [ ]:
# Adicionar import do pandas
import pandas as pd
from pathlib import Path

# Carregar o CSV para análise
df_final = pd.read_csv(output_path)

# Gerar resumo
summary = consolidator.get_data_summary(df_final)

print("\n📊 RESUMO DOS DADOS CONSOLIDADOS:")
print("="*50)
print(f"📝 Total de registros: {summary['total_records']:,}")
print(f"📋 Total de colunas: {summary['total_columns']}")
print(f"💾 Uso de memória: {summary['memory_usage_mb']:.2f} MB")

if 'coordinate_stats' in summary:
    coords = summary['coordinate_stats']
    print(f"\n🗺️ COORDENADAS:")
    print(f"  Latitude: {coords['lat_min']:.6f} → {coords['lat_max']:.6f}")
    print(f"  Longitude: {coords['lon_min']:.6f} → {coords['lon_max']:.6f}")

print(f"\n📂 Colunas disponíveis:")
for i, col in enumerate(summary['columns'][:10], 1):
    print(f"  {i:2d}. {col}")
if len(summary['columns']) > 10:
    print(f"  ... e mais {len(summary['columns']) - 10} colunas")

# Verificar tamanho do arquivo
file_size = Path(output_path).stat().st_size / 1024 / 1024
print(f"\n💾 Tamanho do CSV: {file_size:.2f} MB")